In [1]:
# Imports
from ast import literal_eval
from os import chdir, walk

import dask.dataframe as dd
import numpy as np
import pandas as pd
from dask.distributed import LocalCluster
import polars as pl
from IPython.display import clear_output
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
paths = []

# Gets paths to review JSONs by appending file name to name of containing folder
for dir, _, items in walk('../../data/tags/'):
    for item in items:
        paths.append(dir+item)

In [3]:
df_tags = pl.read_parquet(paths, allow_missing_columns=True)

In [4]:
df_tags = df_tags.to_pandas()
df_tags

,tagid,name,count,browseable,appid
0,19,Action,51,True,283080
1,21,Adventure,49,True,283080
2,492,Indie,47,True,283080
3,122,RPG,46,True,283080
4,3871,2D,26,True,283080
...,...,...,...,...,...
681218,1721,Psychological Horror,217,True,1862210
681219,599,Simulation,213,True,1862210
681220,4106,Action-Adventure,190,True,1862210
681221,1664,Puzzle,182,True,1862210


In [5]:
df_tags = df_tags.rename(columns={'name':'tag'})
df_tags['tag'] = df_tags['tag'].astype('category')
df_tags = df_tags.drop(['tagid','browseable'], axis=1)
df_tags

,tag,count,appid
0,Action,51,283080
1,Adventure,49,283080
2,Indie,47,283080
3,RPG,46,283080
4,2D,26,283080
...,...,...,...
681218,Psychological Horror,217,1862210
681219,Simulation,213,1862210
681220,Action-Adventure,190,1862210
681221,Puzzle,182,1862210


In [6]:
df_tags = df_tags.set_index(['appid','tag'])
df_tags['ratio'] = df_tags['count'] / df_tags.groupby('appid')['count'].max()
df_tags = df_tags.reset_index()
df_tags

,appid,tag,count,ratio
0,283080,Action,51,1.000000
1,283080,Adventure,49,0.960784
2,283080,Indie,47,0.921569
3,283080,RPG,46,0.901961
4,283080,2D,26,0.509804
...,...,...,...,...
681218,1862210,Psychological Horror,217,0.789091
681219,1862210,Simulation,213,0.774545
681220,1862210,Action-Adventure,190,0.690909
681221,1862210,Puzzle,182,0.661818


In [7]:
# Outer Wilds
df_tags[df_tags['appid'] == 753640].sort_values('ratio', ascending=False).drop('appid', axis=1).style.background_gradient(cmap='viridis', vmin=0)

,tag,count,ratio
285263,Exploration,1143,1.000000
285264,Space,1006,0.880140
285265,Mystery,898,0.785652
285266,Story Rich,871,0.762030
285267,Adventure,821,0.718285
285268,Great Soundtrack,771,0.674541
285269,Atmospheric,755,0.660542
285270,Puzzle,705,0.616798
285271,Open World,670,0.586177
285272,Singleplayer,626,0.547682


In [8]:
def preview_tags(appid: int, name: str):
    df_subset = df_tags[df_tags['appid'] == appid].sort_values('ratio', ascending=False).drop(['appid','count'], axis=1)
    df_subset['importance'] = df_subset['ratio'] > 0.2
    df_subset['importance'] = df_subset['importance'].astype(int) + (df_subset['ratio'] > 0.5)
    print("Tags: " + name)
    display(df_subset.style.background_gradient(subset='ratio', cmap='viridis', vmin=0).background_gradient(subset='importance', cmap='coolwarm', vmin=0))
    

In [9]:
preview_tags(753640, 'Outer Wilds')

Tags: Outer Wilds


,tag,ratio,importance
285263,Exploration,1.000000,2
285264,Space,0.880140,2
285265,Mystery,0.785652,2
285266,Story Rich,0.762030,2
285267,Adventure,0.718285,2
285268,Great Soundtrack,0.674541,2
285269,Atmospheric,0.660542,2
285270,Puzzle,0.616798,2
285271,Open World,0.586177,2
285272,Singleplayer,0.547682,2


In [10]:
preview_tags(653530, 'Return of the Obra Dinn')

Tags: Return of the Obra Dinn


,tag,ratio,importance
666527,Detective,1.000000,2
666528,Mystery,0.950100,2
666529,Puzzle,0.746507,2
666530,Story Rich,0.666667,2
666531,Atmospheric,0.640719,2
666532,Singleplayer,0.610778,2
666533,Indie,0.530938,2
666534,Great Soundtrack,0.403194,1
666535,First-Person,0.323353,1
666536,Historical,0.309381,1


In [11]:
preview_tags(1150690, 'OMORI')

Tags: OMORI


,tag,ratio,importance
79514,Psychological Horror,1.000000,2
79515,Story Rich,0.844787,2
79516,Pixel Graphics,0.721564,2
79517,Indie,0.696682,2
79518,Great Soundtrack,0.672986,2
79519,RPG,0.558452,2
79520,Multiple Endings,0.549763,2
79521,RPGMaker,0.490916,1
79522,Hand-drawn,0.471564,1
79523,Horror,0.458926,1


In [73]:
preview_tags(698780, 'Doki Doki Literature Club!')

Tags: Doki Doki Literature Club!


,tag,ratio,importance
188956,Psychological Horror,1.000000,2
188957,Visual Novel,0.821575,2
188958,Anime,0.816682,2
188959,Free to Play,0.801661,2
188960,Dating Sim,0.627902,2
188961,Horror,0.624374,2
188962,Cute,0.624147,2
188963,Dark,0.508193,2
188964,Story Rich,0.459832,1
188965,Romance,0.443446,1


In [12]:
preview_tags(391540, 'Undertale')

Tags: Undertale


,tag,ratio,importance
418945,Great Soundtrack,1.000000,2
418946,Story Rich,0.906951,2
418947,Choices Matter,0.829683,2
418948,Multiple Endings,0.794067,2
418949,Pixel Graphics,0.789238,2
418950,Funny,0.744912,2
418951,RPG,0.719214,2
418952,Singleplayer,0.705674,2
418953,Indie,0.615816,2
418954,Comedy,0.521645,2


In [13]:
preview_tags(646570, 'Slay the Spire')

Tags: Slay the Spire


,tag,ratio,importance
642194,Roguelike Deckbuilder,1.000000,2
642195,Card Game,0.918728,2
642196,Card Battler,0.892226,2
642197,Roguelike,0.793286,2
642198,Deckbuilding,0.646054,2
642199,Turn-Based,0.570671,2
642200,Strategy,0.461131,1
642201,Singleplayer,0.393993,1
642202,Roguelite,0.373380,1
642203,Replay Value,0.313310,1


In [14]:
preview_tags(774361, "Blasphemous")

Tags: Blasphemous


,tag,ratio,importance
120456,Metroidvania,1.000000,2
120457,Pixel Graphics,0.808605,2
120458,Souls-like,0.805638,2
120459,Dark Fantasy,0.645401,2
120460,Gore,0.636499,2
120461,2D,0.563798,2
120462,Violent,0.519288,2
120463,Action,0.428783,1
120464,Indie,0.424332,1
120465,Difficult,0.415430,1


In [63]:
#df_tags['importance'] = (df_tags['ratio'] > 0.2).astype(int) + ((df_tags['ratio'] > 0.5).astype(int)*2**0.5-1)
df_tags['importance'] = (df_tags['ratio'] > 0.2).astype(int) + (df_tags['ratio'] > 0.5).astype(int)

In [64]:
df_tags_pivotted = pd.pivot_table(df_tags[(df_tags['importance'] > 0) & (df_tags['tag'] != 'Indie')][['appid','tag', 'importance']], index='appid', columns='tag', observed=True)
df_tags_pivotted = df_tags_pivotted['importance'].fillna(0).astype(int)
df_tags_pivotted

tag,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
appid,,,,,,,,,,,,,,,,,,,,,
1002,0,0,0,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1500,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1510,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289130,0,0,0,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3289230,0,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3293040,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_games = pd.read_parquet("../../data/store_info.parquet").reset_index().rename(columns={'steam_appid':'appid'}).set_index('appid')
df_games

,type,name,required_age,is_free,dlc,detailed_description,about_the_game,short_description,supported_languages,reviews,...,pc_requirements_recommended,mac_requirements_minimum,mac_requirements_recommended,linux_requirements_minimum,linux_requirements_recommended,compatible_windows,compatible_mac,compatible_linux,categories_names,genres_names
appid,,,,,,,,,,,,,,,,,,,,,
1000000,game,ASCENXION,0,False,[1077070],ASCENXION is a 2D shoot 'em up game where you ...,ASCENXION is a 2D shoot 'em up game where you ...,ASCENXION is a 2D shoot 'em up game where you ...,"Korean, English, Simplified Chinese","“게임 피지컬을 발휘해 적의 탄막 패턴을 피하고 일반 공격과 특수 스킬, 무기 스킬...",...,None,None,None,None,None,True,False,False,"[Single-player, Steam Achievements, Partial Co...","[Action, Adventure, Indie]"
1000040,game,细胞战争,0,False,None,Hello everyone I am the developer of this game...,Hello everyone I am the developer of this game...,这是一款打击感十足的细胞主题游戏！操作简单但活下去却不简单，“你”作为侵入人体的细菌病毒，通...,Simplified Chinese,None,...,Requires a 64-bit processor and operating system,,,,,True,False,False,"[Single-player, Family Sharing]","[Action, Casual, Indie, Simulation]"
1000080,game,Zengeon,0,False,"[1153360, 1159380, 1163180, 1166500, 1169220]",Zengeon is a style-bursting colour filled fusi...,Zengeon is a style-bursting colour filled fusi...,Zengeon is an anime infused Action RPG and Rog...,"Simplified Chinese, English, Traditional Chine...",None,...,OS: Windows10 64bit\nProcessor: I7-6700HQ 2.59...,OS: OS: MacOS 10.7\nProcessor: Processor: Dual...,None,None,None,True,True,False,"[Single-player, Multi-player, Co-op, Online Co...","[Action, Adventure, Indie, RPG]"
1000100,game,干支セトラ 陽ノ卷｜干支etc. 陽之卷,0,False,None,セーブデータ連動『干支セトラ 陰ノ巻』のセーブデータがあると連動する部分があります。\n『陽...,主人公と十二支を軸に展開する和風風水伝奇。君たちが紡ぐ四通りの物語。彼らの欠落が呼ぶものは、...,耐用年数を超えて綻びゆく都市風水を修復するため、次代の風水師候補に選ばれた主人公。 綻びから...,"Japanese*, Simplified Chinese, Traditional Chi...",None,...,OS: Windows10 64bit\nMemory: ２ GB RAM\nStorage...,None,None,None,None,True,False,False,"[Single-player, Steam Achievements, Steam Clou...","[Adventure, Indie, RPG, Strategy]"
1000130,game,Cube Defender,0,False,None,Content Expansion #2 Now LiveThe second conten...,Content Expansion #2 Now LiveThe second conten...,Build turrets and destroy wave after wave of c...,English**languages with full audio support,None,...,OS *: Windows 8 or Newer (Windows 8/10 Support...,OS: OS X 10.11\nProcessor: 2.0Ghz\nMemory: 1 G...,OS: OS X 10.13+\nProcessor: 2.9Ghz (dual-core ...,None,None,True,True,False,"[Single-player, Steam Achievements, Steam Lead...","[Casual, Indie]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999830,game,Becalm,0,True,None,In this relaxing first person voyage you'll sa...,In this relaxing first person voyage you'll sa...,Fancy a relaxing drift through magical waters?...,"English, French, Italian, German, Spanish - Sp...","“Sail the cosmically coloured waters, take in ...",...,Requires a 64-bit processor and operating system,OS: Mac OS X 10.8 or Later\nProcessor: Intel C...,,OS: Ubuntu 12.04 or later\nProcessor: 1.5 Ghz ...,,True,True,True,"[Single-player, Full controller support]","[Casual, Free To Play, Indie]"
999860,game,Enemy On Board,0,True,[1287770],Enemy On Board is an online multiplayer (6vs2)...,Enemy On Board is an online multiplayer (6vs2)...,Online (6v2) social deduction game that will r...,English**languages with full audio support,None,...,OS *: Windows 7+\nProcessor: Any dual core pro...,OS: Sierra 10.12.6+\nProcessor: Any dual core ...,OS: Sierra 10.12.6+\nProcessor: Any dual core ...,None,None,True,False,False,"[Multi-player, PvP, Online PvP, Co-op, Online ...","[Action, Casual, Free To Play, Indie, Early Ac..."
999890,game,Bruken,0,False,None,Bruken? That's a weird name... Well when you g...,Bruken? That's a weird name... Well when you g...,"Explore a large spherical world, enjoying all ...",English,“Bruken отличная игра с элементами головоломки...,...,OS: Windows 10\nProcessor: Core i5 3 GHz or eq...,None

In [68]:
similarities = cosine_similarity(df_tags_pivotted.loc[1150690].values.reshape(1,-1), df_tags_pivotted)[0]
df_similarities = pd.DataFrame(zip(df_tags_pivotted.index, similarities), columns=['appid','similarity']).set_index('appid')
df_similarities['name'] = df_games['name']
df_similarities.sort_values('similarity', ascending=False).head(11)

,similarity,name
appid,,
1150690,1.000000,OMORI
524850,0.733946,Alicemare
373770,0.673451,LiEat
887300,0.665950,Bleeding Sun
1826060,0.641167,Night Loops
741250,0.631474,The Sand Man
2399910,0.626013,Byakuya Museum
706560,0.609872,Jimmy and the Pulsating Mass
1110020,0.609872,Image of Perfection


In [69]:
from cuml import UMAP
umap = UMAP(n_components = int(df_tags_pivotted.shape[1]**0.5), n_neighbors = int(df_tags_pivotted.shape[0]**(1/3)))
df_tags_umap = umap.fit_transform(df_tags_pivotted)
df_tags_umap

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
appid,,,,,,,,,,,,,,,,,,,,,
1002,1.336315,-1.669176,0.404609,0.482137,-0.032838,-0.885944,1.312122,0.994090,-1.849953,0.465027,...,-1.440328,-1.168711,-1.407056,-1.255291,2.125671,-1.142427,-0.238710,0.749965,-1.604308,0.018444
1500,-0.026833,-0.708125,0.207348,-0.100837,0.213391,-1.979029,1.851932,1.389140,0.898735,-0.816996,...,0.059140,-1.317009,-0.624195,0.128357,0.979807,-0.803962,-0.990884,-0.283867,-2.202190,-0.420876
1510,0.080351,-0.694960,0.670839,-0.141827,0.117786,-2.113686,2.117403,0.936072,0.586385,-0.603949,...,0.053032,-0.880790,-0.105335,0.118249,0.761271,-0.720694,-1.200045,-0.836699,-1.780892,-0.050096
2400,1.166371,-1.740814,-0.295580,0.683224,-0.389312,-0.986382,1.443798,0.995741,-2.099030,0.376746,...,-1.942698,-1.207727,-1.326247,-1.367011,2.248900,-1.513803,-0.142344,0.452755,-1.749077,0.618448
2420,1.514526,-2.311695,-0.552753,1.030354,-0.044033,-0.727955,1.633911,1.710556,-2.696426,0.551346,...,-2.624207,-1.362450,-1.831421,-1.925903,2.586357,-2.465851,0.062691,0.838524,-2.036316,0.878189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289130,0.129798,-0.702516,0.837134,-0.202060,0.600183,-1.638072,1.781156,1.549070,1.006007,-0.426334,...,0.156631,-1.289596,-1.168269,0.057895,0.561003,-0.786543,-1.059197,0.066500,-1.837682,-0.990732
3289230,0.451279,-0.835490,1.654900,-0.083445,0.517198,-1.040751,1.874045,0.593287,-0.220503,0.841302,...,-0.514022,-0.278868,-0.868967,-0.355280,-0.217889,-0.985548,-1.396788,-0.513359,-0.353768,-0.523730
3293040,-0.322487,-0.406303,0.478649,-0.094781,-0.182528,-0.982750,1.520667,0.006436,0.088796,0.448544,...,-0.737449,-0.225069,-0.439801,0.101895,-0.327642,-0.916355,-1.410734,-0.960020,-0.317262,-0.142090


In [70]:
similarities = cosine_similarity(df_tags_umap.loc[1150690].values.reshape(1,-1), df_tags_umap)[0]
df_similarities = pd.DataFrame(zip(df_tags_umap.index, similarities), columns=['appid','similarity']).set_index('appid')
df_similarities['name'] = df_games['name']
df_similarities.sort_values('similarity', ascending=False).head(11)

,similarity,name
appid,,
1150690,1.000000,OMORI
1632880,0.999627,学院逃生
2355220,0.999547,The Twin Paradox
1740230,0.999477,Furquest
2477130,0.999441,Märchen Kanna
1392460,0.999404,Project of the Gods
1561910,0.999374,凯尔文 - Kelvin And The Chateau
652270,0.999114,Dark and Bright
681100,0.999100,Supermagical


In [71]:
from cuml import PCA
pca = PCA(n_components = int(df_tags_pivotted.shape[1]**0.5))
df_tags_pca = pca.fit_transform(df_tags_pivotted)
df_tags_pca

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
appid,,,,,,,,,,,,,,,,,,,,,
1002,-1.077584,-0.093542,-1.123262,0.218216,-0.479671,0.480217,0.562102,-0.170819,-0.156442,-0.150235,...,0.415956,0.241166,-0.207379,1.684249,0.411193,0.417159,0.332361,-0.536197,0.015933,-0.559938
1500,0.886938,-1.223714,-0.703157,1.157953,-2.275692,-0.094906,0.933596,1.127317,-0.737168,-0.063069,...,-0.448440,-1.741993,-0.541717,-1.039358,-1.967275,-0.063771,-0.254538,0.090667,0.914754,0.211063
1510,1.057582,-0.340775,-0.014832,-0.123725,-2.671515,-0.198646,0.508064,-1.189621,0.562516,-0.868077,...,-0.016021,-0.590271,-1.064919,0.205393,-0.317913,0.523663,0.216157,0.116355,-0.470839,0.020650
2400,-1.091623,0.948971,0.160075,-0.004798,-0.234793,0.166536,0.652754,0.265170,-0.463606,0.608645,...,0.740685,0.649010,0.678257,0.668769,-0.412545,0.250788,0.115488,0.086938,0.146283,0.160612
2420,-1.463829,1.096739,-0.436695,-0.152604,0.323398,0.064442,0.822556,-0.076955,-0.323480,-0.103240,...,0.302349,0.233822,-0.522325,-0.225713,0.168590,-0.044082,-0.068191,-0.222446,0.149679,-0.197750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3289130,1.282092,0.788624,-0.691048,-0.718240,-0.449421,-0.529313,0.080311,0.081301,0.547284,-0.549388,...,-0.360776,0.724749,-0.085755,0.008501,0.187445,0.041378,0.118883,0.553175,-0.275070,-0.612489
3289230,2.623539,-0.381036,-1.322959,-0.224537,-0.525835,-0.113238,-0.766950,-1.596845,0.000753,0.529836,...,0.837267,-0.016755,0.549128,-0.662153,1.189746,-0.382657,-0.776557,0.464839,0.214581,0.158344
3293040,0.827801,1.618538,0.721439,0.380913,-0.488041,0.412026,-1.123063,-0.338167,0.227710,1.105170,...,-0.741883,0.737550,0.053463,-0.218967,-0.827105,-0.005920,-0.646605,0.544130,-0.630544,0.576530


In [72]:
similarities = cosine_similarity(df_tags_pca.loc[1150690].values.reshape(1,-1), df_tags_pca)[0]
df_similarities = pd.DataFrame(zip(df_tags_pca.index, similarities), columns=['appid','similarity']).set_index('appid')
df_similarities['name'] = df_games['name']
df_similarities.sort_values('similarity', ascending=False).head(11)

,similarity,name
appid,,
1150690,1.000000,OMORI
2399910,0.939395,Byakuya Museum
2726320,0.901129,Меліній
373770,0.896844,LiEat
1949420,0.872828,Ren's Demons I
3238160,0.858316,oct 14
1687550,0.833305,Bear's Restaurant
1221730,0.829389,Anti-Science Brigade
2604880,0.826824,Panoptia
